In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import gbd_mapping as gbd
from db_queries import get_outputs, get_ids
from get_draws.api import get_draws
from vivarium import Artifact

import bep_summarizer

!whoami
!date

ndbs
Wed Feb  5 12:08:59 PST 2020


In [2]:
%load_ext autoreload
%autoreload 2

## Output directories

In [3]:
artifact_dir = '/ihme/costeffectiveness/artifacts/vivarium_gates_bep/'
results_dir = '/share/costeffectiveness/results/vivarium_gates_bep/base_model_half_day'

locations_rundates = {
    'Mali': '2020_02_04_20_04_06',
    'Tanzania': '2020_02_04_20_09_12'
}

locations = list(locations_rundates.keys())

## GBD IDs from R shiny tool

https://shiny.ihme.washington.edu/content/88/

(Can also use `get_ids`)

In [4]:
# GBD round IDs
gbd_2017 = 5

# Location IDs
tanzania = 189 # United Republic of Tanzania 
mali = 211 # Mali

# Cause IDs
all_causes = 294
diarrhea = 302 # Diarrheal diseases
lri = 322 # Lower respiratory infections
meningitis = 332
measles = 341
neonatal = 380 # Neonatal disorders
pem = 387 # Protein energy malnutrition

# Measures
deaths = 1
prevalence = 5
incidence = 6
emr = 9 # Excess mortality rate
acmr = 14 # All-cause mortality rate
csmr = 15 # Cause-specific mortality rate
relative_risk = 11
birth_prev = 38 # Birth prevalence

# Metrics
number = 1
percent = 2
rate = 3

# Age group IDs
at_birth = 164
early_nn = 2
late_nn = 3
post_nn = 4
one_to_four = 5

# Sex IDs
male=1
female=2
both=3

## Read in the data artifact for Mali

In [5]:
art_mali = Artifact(artifact_dir + 'mali.hdf', filter_terms=['year_start == 2017', 'age_start < 5'])
print(art_mali)

Artifact containing the following keys:
metadata
	keyspace
	locations
population
	structure
	age_bins
	demographic_dimensions
	theoretical_minimum_risk_life_expectancy
cause
	all_causes
		cause_specific_mortality_rate
	diarrheal_diseases
		prevalence
		incidence_rate
		remission_rate
		cause_specific_mortality_rate
		excess_mortality_rate
		disability_weight
		restrictions
	measles
		prevalence
		incidence_rate
		cause_specific_mortality_rate
		excess_mortality_rate
		disability_weight
		restrictions
	lower_respiratory_infections
		prevalence
		incidence_rate
		remission_rate
		cause_specific_mortality_rate
		excess_mortality_rate
		disability_weight
		restrictions
	meningitis
		prevalence
		incidence_rate
		remission_rate
		cause_specific_mortality_rate
		excess_mortality_rate
		disability_weight
		restrictions
	protein_energy_malnutrition
		cause_specific_mortality_rate
		excess_mortality_rate
		disability_weight
		restrictions
	neonatal_disorders
		cause_specific_mortality_rate
		pr

## See what's stored in LBWSG risk

Not much - only metadata

In [6]:
art_mali.load('risk_factor.low_birth_weight_and_short_gestation.distribution')

'ordered_polytomous'

In [7]:
art_mali.load('risk_factor.low_birth_weight_and_short_gestation.categories')

{'cat2': 'Birth prevalence - [0, 24) wks, [0, 500) g',
 'cat8': 'Birth prevalence - [0, 24) wks, [500, 1000) g',
 'cat10': 'Birth prevalence - [24, 26) wks, [500, 1000) g',
 'cat11': 'Birth prevalence - [26, 28) wks, [500, 1000) g',
 'cat14': 'Birth prevalence - [30, 32) wks, [500, 1000) g',
 'cat15': 'Birth prevalence - [28, 30) wks, [500, 1000) g',
 'cat17': 'Birth prevalence - [26, 28) wks, [1000, 1500) g',
 'cat19': 'Birth prevalence - [34, 36) wks, [1000, 1500) g',
 'cat20': 'Birth prevalence - [28, 30) wks, [1500, 2000) g',
 'cat21': 'Birth prevalence - [28, 30) wks, [1000, 1500) g',
 'cat22': 'Birth prevalence - [32, 34) wks, [1000, 1500) g',
 'cat23': 'Birth prevalence - [30, 32) wks, [1000, 1500) g',
 'cat24': 'Birth prevalence - [37, 38) wks, [1500, 2000) g',
 'cat25': 'Birth prevalence - [36, 37) wks, [1500, 2000) g',
 'cat26': 'Birth prevalence - [30, 32) wks, [2000, 2500) g',
 'cat27': 'Birth prevalence - [30, 32) wks, [1500, 2000) g',
 'cat28': 'Birth prevalence - [34, 36

## See what's in the child wasting risk

This one actually has data, namely relative risks

In [10]:
art_mali.load('risk_factor.child_wasting.distribution')

'ordered_polytomous'

In [8]:
art_mali.load('risk_factor.child_wasting.categories')

{'cat1': 'Child wasting exposure, <-3 sd',
 'cat2': 'Child wasting exposure,  -3 to -2 sd',
 'cat3': 'Child wasting exposure,  -2 to -1 sd',
 'cat4': 'Unexposed'}

In [9]:
art_mali.load('risk_factor.child_wasting.relative_risk')

draw_0  \
location sex    age_start age_end  year_start year_end affected_entity              affected_measure parameter               
Mali     Female 0.000000  0.019178 2017       2018     diarrheal_diseases           incidence_rate   cat1         1.000000   
                                                                                                     cat2         1.000000   
                                                                                                     cat3         1.000000   
                                                                                                     cat4         1.000000   
                                                       lower_respiratory_infections incidence_rate   cat1         1.000000   
                                                                                                     cat2         1.000000   
                                                                                                     cat3         1.000000   
                                                                                                     cat4         1.000000   
                                                       measles                      incidence_rate   cat1         1.000000   
                                                                                                     cat2         1.000000   
                                                                                                     cat3         1.000000   
                                                                                                     cat4         1.000000   
                0.019178  0.076712 2017       2018     diarrheal_diseases           incidence_rate   cat1         1.000000   
                                                                                                     cat2         1.000000   
                                                                                                     cat3         1.000000   
                                                                                                     cat4         1.000000   
                                                       lower_respiratory_infections incidence_rate   cat1         1.000000   
                                                                                                     cat2         1.000000   
                                                                                                     cat3         1.000000   
                                                                                                     cat4         1.000000   
                                                       measles                      incidence_rate   cat1         1.000000   
                                                                                                     cat2         1.000000   
                                                                                                     cat3         1.000000   
                                                                                                     cat4         1.000000   
                0.076712  1.000000 2017       2018     diarrheal_diseases           incidence_rate   cat1        69.619331   
                                                                                                     cat2        17.917391   
                                                                                                     cat3         4.735045   
                                                                                                     cat4         1.000000   
                                                       lower_respiratory_infections incidence_rate   cat1        63.121151   
                                                                                                     cat2        29.701700   
...                                                                                 

## The child wasting 'alternative risk factor' is the WHZ distribution

This has means, standard deviations, and ensemble weights

In [11]:
art_mali.load('alternative_risk_factor.child_wasting.distribution')

'ensemble'

In [12]:
art_mali.load('alternative_risk_factor.child_wasting.exposure')

draw_0  \
location sex    age_start age_end  year_start year_end parameter              
Mali     Female 0.000000  0.019178 2017       2018     continuous  9.480126   
                0.019178  0.076712 2017       2018     continuous  9.639698   
                0.076712  1.000000 2017       2018     continuous  9.640467   
                1.000000  5.000000 2017       2018     continuous  9.584609   
         Male   0.000000  0.019178 2017       2018     continuous  9.348635   
                0.019178  0.076712 2017       2018     continuous  9.370745   
                0.076712  1.000000 2017       2018     continuous  9.409954   
                1.000000  5.000000 2017       2018     continuous  9.367189   

                                                                     draw_1  \
location sex    age_start age_end  year_start year_end parameter              
Mali     Female 0.000000  0.019178 2017       2018     continuous  9.451813   
                0.019178  0.076712 2017       2018     continuous  9.564015   
                0.076712  1.000000 2017       2018     continuous  9.740894   
                1.000000  5.000000 2017       2018     continuous  9.484688   
         Male   0.000000  0.019178 2017       2018     continuous  9.260438   
                0.019178  0.076712 2017       2018     continuous  9.326985   
                0.076712  1.000000 2017       2018     continuous  9.389954   
                1.000000  5.000000 2017       2018     continuous  9.364774   

                                                                     draw_2  \
location sex    age_start age_end  year_start year_end parameter              
Mali     Female 0.000000  0.019178 2017       2018     continuous  9.345155   
                0.019178  0.076712 2017       2018     continuous  9.673154   
                0.076712  1.000000 2017       2018     continuous  9.763652   
                1.000000  5.000000 2017       2018     continuous  9.590926   
         Male   0.000000  0.019178 2017       2018     continuous  9.413945   
                0.019178  0.076712 2017       2018     continuous  9.417479   
                0.076712  1.000000 2017       2018     continuous  9.525630   
                1.000000  5.000000 2017       2018     continuous  9.519881   

                                                                     draw_3  \
location sex    age_start age_end  year_start year_end parameter              
Mali     Female 0.000000  0.019178 2017       2018     continuous  9.321670   
                0.019178  0.076712 2017       2018     continuous  9.626109   
                0.076712  1.000000 2017       2018     continuous  9.681860   
                1.000000  5.000000 2017       2018     continuous  9.453885   
         Male   0.000000  0.019178 2017       2018     continuous  9.315665   
                0.019178  0.076712 2017       2018     continuous  9.470984   
                0.076712  1.000000 2017       2018     continuous  9.293487   
                1.000000  5.000000 2017       2018     continuous  9.292613   

                                                                     draw_4  \
location sex    age_start age_end  year_start year_end parameter              
Mali     Female 0.000000  0.019178 2017       2018     continuous  9.484431   
                0.019178  0.076712 2017       2018     continuous  9.590132   
                0.076712  1.000000 2017       2018     continuous  9.789382   
                1.000000  5.000000 2017       2018     continuous  9.643395   
         Male   0.000000  0.019178 2017       2018     continuous  9.413945   
                0.019178  0.076712 2017       2018     continuous  9.558673   
                0.076712  1.000000 2017       2018     continuous  9.415859   
                1.000000  5.000000 2017       2018     continuous  9.376088   

                                                                     draw_5  \
location sex    age_start age_end  

In [13]:
art_mali.load('alternative_risk_factor.child_wasting.exposure_standard_deviation')

draw_0    draw_1  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      1.621279  1.437984   
                0.019178  0.076712 2017       2018      1.504652  1.624165   
                0.076712  1.000000 2017       2018      1.203166  1.262259   
                1.000000  5.000000 2017       2018      1.434674  1.346596   
         Male   0.000000  0.019178 2017       2018      1.612376  1.525234   
                0.019178  0.076712 2017       2018      1.206516  1.258144   
                0.076712  1.000000 2017       2018      1.388969  1.433863   
                1.000000  5.000000 2017       2018      1.372464  1.310867   

                                                          draw_2    draw_3  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      1.362859  1.401444   
                0.019178  0.076712 2017       2018      1.779417  1.241534   
                0.076712  1.000000 2017       2018      1.517477  1.066141   
                1.000000  5.000000 2017       2018      1.396098  1.242925   
         Male   0.000000  0.019178 2017       2018      1.720410  1.588501   
                0.019178  0.076712 2017       2018      1.558980  1.799884   
                0.076712  1.000000 2017       2018      1.728202  1.229209   
                1.000000  5.000000 2017       2018      1.516798  1.280802   

                                                          draw_4    draw_5  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      1.469768  1.484494   
                0.019178  0.076712 2017       2018      1.862747  1.511708   
                0.076712  1.000000 2017       2018      1.238895  1.471510   
                1.000000  5.000000 2017       2018      1.412417  1.430506   
         Male   0.000000  0.019178 2017       2018      1.720352  1.621767   
                0.019178  0.076712 2017       2018      1.519979  1.322057   
                0.076712  1.000000 2017       2018      1.366688  1.831808   
                1.000000  5.000000 2017       2018      1.401942  1.545544   

                                                          draw_6    draw_7  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      1.354836  1.490241   
                0.019178  0.076712 2017       2018      1.297603  1.605558   
                0.076712  1.000000 2017       2018      1.252911  1.237685   
                1.000000  5.000000 2017       2018      1.415096  1.225576   
         Male   0.000000  0.019178 2017       2018      1.738587  1.685550   
                0.019178  0.076712 2017       2018      1.226662  1.163898   
                0.076712  1.000000 2017       2018      1.486271  0.981783   
                1.000000  5.000000 2017       2018      1.109243  1.307138   

                                                          draw_8    draw_9  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      1.649878  1.545574   
                0.019178  0.076712 2017       2018      1.205617  1.361969   
                0.076712  1.000000 2017       2018      1.274022  1.268541   
                1.000000  5.000000 2017       2018      1.291411  1.253092   
         Male   0.000000  0.019178 2017       2018      1.544750  1.475197   
                0.019178  0.076712 2017       2018      1.107379  1.658940   
                0.076712  1.000000 2017       2018      1.574077  1.489666   
                1.000000  5.000000 2017       2018      1.289672  1.389293   

                                                        ...  draw_990  \
location sex    age_start age_end  year_start year_end  ...             
Mali   

In [14]:
art_mali.load('alternative_risk_factor.child_wasting.exposure_distribution_weights')

value
location sex    age_start age_end  year_start year_end parameter           
Mali     Female 0.000000  0.019178 2017       2018     betasr      0.024588
                                                       exp         0.000966
                                                       gamma       0.009528
                                                       glnorm      0.000000
                                                       gumbel      0.009508
                                                       invgamma    0.005451
                                                       invweibull  0.029552
                                                       llogis      0.462327
                                                       lnorm       0.007952
                                                       mgamma      0.095179
                                                       mgumbel     0.107609
                                                       norm        0.027990
                                                       weibull     0.219351
                0.019178  0.076712 2017       2018     betasr      0.024588
                                                       exp         0.000966
                                                       gamma       0.009528
                                                       glnorm      0.000000
                                                       gumbel      0.009508
                                                       invgamma    0.005451
                                                       invweibull  0.029552
                                                       llogis      0.462327
                                                       lnorm       0.007952
                                                       mgamma      0.095179
                                                       mgumbel     0.107609
                                                       norm        0.027990
                                                       weibull     0.219351
                0.076712  1.000000 2017       2018     betasr      0.024588
                                                       exp         0.000966
                                                       gamma       0.009528
                                                       glnorm      0.000000
...                                                                     ...
         Male   0.019178  0.076712 2017       2018     mgamma      0.095179
                                                       mgumbel     0.107609
                                                       norm        0.027990
                                                       weibull     0.219351
                0.076712  1.000000 2017       2018     betasr      0.024588
                                                       exp         0.000966
                                                       gamma       0.009528
                                                       glnorm      0.000000
                                                       gumbel      0.009508
                                                       invgamma    0.005451
                                                       invweibull  0.029552
                                                       llogis      0.462327
                                                       lnorm       0.007952
                                                       mgamma      0.095179
                                                       mgumbel     0.107609
                                                       norm        0.027990
                                                       weibull     0.219351
                1.000000  5.000000 2017       2018     betasr      0.024588
                                                       exp         0.000966
                                                       gamma       0.009528
                                          